In [ ]:
from scipy.stats import ttest_ind
import helper 
import xarray as xr 
import numpy as np 
import cmaps as cmap 
import matplotlib.pyplot as plt
import scipy 
import warnings
import matplotlib.patheffects as pe
from matplotlib.colors import Normalize
from matplotlib import rcParams
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Rectangle
import matplotlib.patheffects as pe
warnings.filterwarnings("ignore")

def rePoPolar(dataset, name, offset):
    x = dataset.lon.values / 4
    y = dataset.lat.values / 4
    x, y = np.meshgrid(x, y)

    R = 6371.0
    cphi = np.cos(np.deg2rad(y))
    x = R * np.deg2rad(x) * cphi
    y = R * np.deg2rad(y)

    r = np.sqrt(x**2 + y**2)
    t = np.arctan2(y, x)

    rBins = np.linspace(np.nanmin(r), np.nanmax(r), 120)
    tBins = np.linspace(np.nanmin(t), np.nanmax(t), 360)

    # for i in range(len(tBins)):
    #         tBins[i] = tBins[i] + offset
    #         while tBins[i] <= (-1 * np.pi):
    #             tBins[i] = tBins[i] + (2 * np.pi)
    #         while tBins[i] >= np.pi:
    #             tBins[i] = tBins[i] - (2 * np.pi)

    R, T = np.meshgrid(rBins, tBins)
    newX, newY = R * np.cos(T), R * np.sin(T)
    gridded_data = scipy.interpolate.griddata((x.flatten(), y.flatten()), dataset.values.flatten(), (newX.flatten(), newY.flatten()), method='nearest')

    polar = xr.Dataset(
        {
            name: (('r', 'theta'), gridded_data.reshape(R.shape).transpose())
        },
        coords={
            'r': rBins,
            'theta': tBins
        }
    )

    return polar


In [ ]:
dataset = xr.open_dataset(r"C:\Users\deela\Downloads\TCRADAR_ERA5.nc")
print(dataset)
print(list(dataset.variables))

In [ ]:
test = dataset.where(dataset.vmax > 130)
print(test)

In [ ]:
def process(data, ddir):
    dataList = []
    print(len(data.case))
    for x in range(len(data.case)):
        # print(x)
        offset = np.deg2rad(360 - ddir.isel(case = x))
        try:
            temp = rePoPolar(data.isel(case = x), 'rh', offset = offset)
        except:
            temp = xr.Dataset(
                {
                    var: (temp[var].dims, np.full_like(temp[var], np.nan))
                    for var in temp.data_vars
                },
                coords=temp.coords,
                attrs=temp.attrs)

        dataList.append(temp)

    newTheta = np.linspace(-np.pi, np.pi, 360, endpoint=False)
    for x in range(len(dataList)):
        try:
            dataList[x] = dataList[x].interp(theta = newTheta)    
            dataList[x]['rh'].values = np.flip(dataList[x]['rh'].values, axis = 1)
        except:
            pass

    rh = xr.concat(dataList, dim = 'case')
    print(rh)  

    return rh, newTheta 

In [ ]:
alignment = [225, 251, 252, 253, 254, 333, 334, 347, 374, 376, 377, 407, 408, 409, 410, 413, 414, 603, 604, 605, 672, 712, 719, 752, 765, 864, 878, 879, 939, 941, 957, 968, 969, 970, 971, 1057, 1073, 1101, 1128, 1131, 1148, 1177, 1178, 1179, 1180, 1191, 1192, 1220, 1221, 1222, 1223, 1224, 1226, 1227, 1228, 1302, 1379, 1380, 1391, 1405, 1406, 1445, 1446, 1447, 1448, 1453, 1471]
misalign = [148, 149, 223, 224, 339, 340, 341, 342, 343, 344, 382, 383, 384, 386, 400, 402, 423, 424, 425, 426, 427, 429, 430, 431, 545, 600, 601, 742, 744, 745, 747, 757, 760, 869, 898, 899, 918, 919, 930, 934, 935, 936, 1042, 1052, 1174, 1175, 1195, 1197, 1201, 1217, 1218, 1372, 1373, 1376, 1377, 1408, 1410, 1418, 1419]

In [ ]:
uData = dataset['u_data']
vData = dataset['v_data']
ddir = dataset['shear_dir']

In [ ]:
levels = range(200, 900, 50)
l = []
for hgt in levels:
    print('Level: ', hgt)
    ud = uData.sel(level = hgt)
    rh, newTheta = process(ud, ddir)
    l.append(rh)
uDataAll = xr.concat(l, dim = 'level')

In [ ]:
levels = range(200, 900, 50)
l = []
for hgt in levels:
    print('Level: ', hgt)
    vd = vData.sel(level = hgt)
    rh, newTheta = process(vd, ddir)
    l.append(rh)
vDataAll = xr.concat(l, dim = 'level')

In [ ]:
print(uDataAll, vDataAll)

In [ ]:
uAlign = uDataAll['rh'].sel(r = slice(0, 500), case = alignment).mean(['r', 'theta', 'case'])
vAlign = vDataAll['rh'].sel(r = slice(0, 500), case = alignment).mean(['r', 'theta', 'case'])

uMisalign = uDataAll['rh'].sel(r = slice(0, 500), case = misalign).mean(['r', 'theta', 'case'])
vMisalign = vDataAll['rh'].sel(r = slice(0, 500), case = misalign).mean(['r', 'theta', 'case'])

In [ ]:
def setUpHodo(max, meanU, meanV):
    fig = plt.figure(figsize=(9.5, 8))

    ax = plt.axes()
    ax.spines[['left', 'bottom']].set_position('zero')
    ax.spines[['top', 'right']].set_visible(False)
    ax.set_frame_on(False)
    ax.grid(linewidth = 0.5, color = 'black', alpha = 0.5, linestyle = '--', zorder = 10)
    ax.axvline(x = 0, c = 'black', zorder = 0)
    ax.axhline(y = 0, c = 'black', zorder = 0)

    if max > 50:
        interval = 10
    elif max > 100:
        interval = 20
    else:
        interval = 5
    max = int(max + interval * 7)
    remainder = max % interval
    max = max - remainder

    for x in range(interval, max + interval, interval):
        c = plt.Circle((0, 0), radius = x, facecolor = "None", edgecolor = '#404040', linestyle = '--')
        ax.add_patch(c)

    ax.set_xlim(meanU - (max / 2), meanU + (max / 2))
    ax.set_ylim(meanV - (max / 2), meanV + (max / 2))

    return ax

In [ ]:
pres = dataset.level.values

In [ ]:
# uAlign = uAlign * 1.94384
# vAlign = vAlign * 1.94384
# uMisalign = uMisalign * 1.94384
# vMisalign = vMisalign * 1.94384

In [ ]:
print(uAlign)

In [ ]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

def pressure2():
    newcmp = LinearSegmentedColormap.from_list("", [
    (0/100, "#FFFFFF"),
    (20/100, "#FFFFFF"),

    (20/100, "#1C1751"),
    (35/100, "#623BAA"),
    (50/100, "#8d74a8"),

    (50/100, "#e2809f"),
    (70/100, "#da3535"),
    (85/100, "#402820"),

    (85/100, "#404040"),
    (100/100, "#000000")])
    
    return newcmp

In [ ]:
ax = setUpHodo(np.nanmax((uAlign**2 + vAlign**2)**0.5), np.mean(uAlign), np.mean(vAlign))
c1 = ax.scatter(uAlign, vAlign, c = pres, linewidth = .5, vmin = 0, vmax = 1000, cmap = cmap.pressure(), zorder = 12)
colors = [cmap.pressure()(pres[l] / 1000) for l in range(len(pres) - 1)]
[ax.plot([u1, u2], [v1, v2], linewidth = 3, color = c, zorder = 11) for (u1, u2, v1, v2, c) in zip(uAlign[:-1], uAlign[1:], vAlign[:-1], vAlign[1:], colors)]

c2 = ax.scatter(uMisalign, vMisalign, c = pres, linewidth = .5, vmin = 0, vmax = 1000, cmap = pressure2(), zorder = 12)
colors = [pressure2()(pres[l] / 1000) for l in range(len(pres) - 1)]
[ax.plot([u1, u2], [v1, v2], linewidth = 3, color = c, zorder = 11) for (u1, u2, v1, v2, c) in zip(uMisalign[:-1], uMisalign[1:], vMisalign[:-1], vMisalign[1:], colors)]


# try:
#     srh = storm_relative_helicity(gphgt * units.meters, uData * units.knots, vData * units.knots, depth = 3 * units.km, storm_u = uMotion * units.knots, storm_v = vMotion * units.knots)
#     ax.text(0.05, 0.05, f'0-3km SRH: {str(round(srh[2].magnitude, 1))}', bbox={'facecolor': 'white', 'edgecolor': 'None', 'alpha': 0.75}, transform = ax.transAxes)
#     ax.scatter(uMotion, vMotion, s = 150, color = 'black', zorder = 20)
#     ax.text(uMotion, vMotion, 'SM', horizontalalignment = 'center', verticalalignment = 'center', color = 'white', size = 6, zorder = 20)
# except:
#     srh = storm_relative_helicity(gphgt * units.meters, uData * units.knots, vData * units.knots, depth = 3 * units.km, storm_u = rM[0], storm_v = rM[1])
#     ax.text(0.05, 0.05, f'0-3km SRH: {str(round(srh[2].magnitude, 1))}', bbox={'facecolor': 'white', 'edgecolor': 'None', 'alpha': 0.75}, transform = ax.transAxes)
#     ax.scatter(rM[0], rM[1], s = 150, color = 'black', zorder = 20)
#     ax.text(rM[0], rM[1], 'rM', horizontalalignment = 'center', verticalalignment = 'center', color = 'white', size = 6, zorder = 20)
#     ax.scatter(lM[0], lM[1], s = 150, color = 'black', zorder = 20)
#     ax.text(lM[0], lM[1], 'lM', horizontalalignment = 'center', verticalalignment = 'center', color = 'white', size = 6, zorder = 20)

ax.set_title(f'TC-RADAR: Aligning vs Non-Aligning TC Average Hodograph (kt)\n500km Radius, Not Shear-Relative' , fontweight='bold', fontsize=10, loc = 'left')
ax.set_title('0.25\u00b0 ERA5\nData from TC-PRIMED', fontsize=10, loc='right') 
cbar1 = plt.colorbar(c1, orientation = 'vertical', aspect = 50, pad = .04, fraction = 0.046, ticks = [200, 500, 700, 850, 1000], label = 'Aligning ')    
cbar1.ax.set_yticklabels([200, 500, 700, 850, 1000])
cbar1.ax.invert_yaxis()

cbar2 = plt.colorbar(c2, orientation = 'vertical', aspect = 50, pad = .02, fraction = 0.046, ticks = [200, 500, 700, 850, 1000], label = 'Non-Aligning')    
cbar2.ax.set_yticklabels(['', '', '', '', ''])
cbar2.ax.invert_yaxis()

plt.savefig(r"C:\Users\deela\Downloads\tcradarhodo.png", dpi = 400, bbox_inches = 'tight')
plt.show() 
plt.close()